# TP : Système de recommandation musicale avec K-means et PCA

Dans ce TP, vous allez découvrir comment combiner deux techniques fondamentales du machine learning :
- **K-means** : algorithme de clustering pour regrouper des morceaux similaires
- **PCA (Analyse en Composantes Principales)** : technique de réduction de dimensionnalité pour visualiser et améliorer le clustering

Vous allez construire un système de recommandation musicale en analysant des caractéristiques audio de morceaux Spotify.

## Plan du TP

1. Exploration des données
2. Première visualisation 3D de features sélectionnées
3. Premier clustering K-means sur données brutes
4. Amélioration avec mise à l'échelle des données
5. Application de la PCA pour réduction de dimensionnalité
6. Clustering optimisé avec K-means sur les composantes principales
7. Détermination du nombre optimal de clusters
8. Création de playlists personnalisées

## 0. Import des bibliothèques

In [ ]:
#!pip install seaborn==0.13.2
#!pip install plotly==6.5.0

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Chargement et exploration des données

### 1.1 Chargement du dataset

In [ ]:
# Charger le dataset - adapter en fonction du dossier où vous avez téléchargé les données
df = pd.read_csv(os.path.join('data','SpotifyTracksDataset','dataset.csv'))

print(f"Dimensions du dataset : {df.shape}")
print(f"Nombre de morceaux : {df.shape[0]}")
print(f"Nombre de colonnes : {df.shape[1]}")

Procédez dans un premier temps à une exploration minimal classique des données (EDA).
Il faudra certainement faire un peu de nettoyage, comme beaucoup de données téléchargées sur Kaggle, etc.

### 1.2 Aperçu des premières lignes

In [ ]:
# VOTRE CODE

On a une colonne `unnamed` qui correspond à l’index visiblement.

In [ ]:
df.set_index('Unnamed: 0',drop=True, inplace=True)
df.index.name = None

### 1.3 Informations sur le dataset

In [ ]:
# VOTRE CODE

### 1.4 Vérification des valeurs manquantes et doublons

In [ ]:
# VOTRE CODE

In [ ]:
# VOTRE CODE

### 1.5 Création d'un DataFrame avec uniquement les données numériques

Pour notre analyse, nous allons nous concentrer sur les features audio quantitatives.

In [ ]:
# Sélection des colonnes numériques pertinentes pour l'analyse audio

data_num = # VOTRE CODE

print(f"Dimensions des données numériques : {data_num.shape}")
print(f"\nFeatures disponibles :")
for i, col in enumerate(data_num.columns, 1):
    print(f"{i}. {col}")

### 1.6 Statistiques descriptives

In [ ]:
# VOTRE CODE

### 1.7 Matrice de corrélation

Analysons les corrélations entre les différentes features audio pour comprendre leurs relations.

In [ ]:
# Calcul de la matrice de corrélation
correlation_matrix = # VOTRE CODE

# Visualisation avec une heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matrice de corrélation des features audio Spotify', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("\n📊 Observations à noter :")
print("- Quelles variables sont fortement corrélées ?")
print("- Y a-t-il des corrélations négatives intéressantes ?")

Lisez bien la description de chaque feature sur la page Kaggle pour d’une part comprendre ces corrélations, et d’autre part sélectionnez trois features (pas forcément corrélées) qu’il vous semble intéressant d’analyser de plus près afin de voir si elles permettraient de regrouper des morceaux ressemblant. Faisons-en une visualisation 3D pour voir comment ces features sont partagées.

Par exemple danceability, energy, valence ou acousticness, instrumentalness, speechiness ou tout autre combinaison qui vous inspire/interpelle.

## 2. Première visualisation 3D avec trois features

Décrivez les 3 features que vous avez choisies :

In [ ]:
# Visualisation 3D des données brutes
fig = px.scatter_3d(data_num, 
                    x= # VOTRE CODE 
                    y=# VOTRE CODE
                    z=# VOTRE CODE
                    opacity=0.7,
                    width=800,
                    height=700,
                    title='Visualisation 3D : Danceability, Energy et Valence (données brutes)')

fig.update_traces(marker=dict(size=3, color='steelblue'))
fig.show()

print("\nObservation : .")


## 3. Premier clustering K-means sur données brutes

### 3.1 Choix du nombre de clusters

Utilisons une règle empirique simple : pour N observations, on peut estimer le nombre de clusters optimal à environ √(N/2). Mais on va éviter d’avoir à considérer plus de 10 clusters (on se pose une limite dans notre exercice, pour se simplifier la vie, on pourrait aussi limiter le nombre d’observations).

In [ ]:
# Calcul du nombre de clusters selon la règle empirique
n_samples = len(data_num)
k_empirical = int(np.sqrt(n_samples / 2))
k_empirical = max(5, min(k_empirical, 10))  # Contraindre entre 5 et 10

print(f"Nombre d'échantillons : {n_samples}")
print(f"Nombre de clusters suggéré (règle empirique) : {k_empirical}")

### 3.2 Application de K-means sur les données brutes

In [ ]:
# K-means sur les données brutes
# VOTRE CODE

### 3.3 Visualisation du clustering sur données brutes

In [ ]:
# Création d'un DataFrame temporaire pour la visualisation
df_viz_raw = # VOTRE CODE

# Visualisation 3D avec les clusters
fig = px.scatter_3d(df_viz_raw,
                    x=# VOTRE CODE
                    y=# VOTRE CODE
                    z=# VOTRE CODE
                    color='cluster',
                    opacity=0.7,
                    width=800,
                    height=700,
                    title=f'K-means sur données brutes ({k_empirical} clusters)')

fig.update_traces(marker=dict(size=3))
fig.show()

print("\nConstat : .")

## 4. Amélioration avec mise à l'échelle des données

### 4.1 Pourquoi scaler les données ?

Les features ont des échelles différentes (ex: duration_ms en millisecondes vs danceability entre 0 et 1). 
Le K-means utilise la distance euclidienne, donc les features avec de grandes valeurs dominent le calcul.

Nous utilisons **RobustScaler** qui est résistant aux outliers (utilise la médiane et l'IQR plutôt que la moyenne).

In [ ]:
# Mise à l'échelle avec RobustScaler
scaler = # VOTRE CODE
data_scaled = # VOTRE CODE

# Création d'un DataFrame pour faciliter l'analyse
data_scaled_df = pd.DataFrame(data_scaled, columns=data_num.columns)

print("Données mises à l'échelle avec RobustScaler")
print(f"\nAperçu des données scalées :")
print(data_scaled_df.describe())

### 4.2 K-means sur les données scalées

In [ ]:
# K-means sur les données scalées
kmeans_scaled = # VOTRE CODE
labels_scaled = # VOTRE CODE

print(f"Clustering effectué avec {k_empirical} clusters sur données scalées")
print(f"Inertie : {kmeans_scaled.inertia_:.2f}")
print(f"\nRépartition des morceaux par cluster :")
unique, counts = np.unique(labels_scaled, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"  Cluster {cluster_id} : {count} morceaux ({count/len(labels_scaled)*100:.1f}%)")

On constate un gros déséquilibre dans la taille (nombre de morceaux) des clusters

### 4.3 Visualisation avec données scalées

In [ ]:
# Visualisation 3D avec les données scalées

# VOTRE CODE

print("\nObservation : ")

## 5. Analyse en Composantes Principales (PCA)

### 5.1 PCA complète pour analyser la variance expliquée

Commençons par faire une PCA sur toutes les composantes possibles pour voir combien de variance est capturée par chacune.

In [ ]:
# PCA complète
pca_full = # VOTRE CODE
# Variance expliquée par chaque composante
variance_explained = # VOTRE CODE
cumulative_variance = # VOTRE CODE

print("Variance expliquée par les premières composantes :")
for i in range(min(10, len(variance_explained))):
    print(f"  PC{i+1} : {variance_explained[i]*100:.2f}% (cumulé : {cumulative_variance[i]*100:.2f}%)")

### 5.2 Graphique de la variance cumulée

In [ ]:
# Visualisation de la variance expliquée
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Variance par composante (fig ax1)
# VOTRE CODE

# Variance cumulée (fig ax2)
# VOTRE CODE

plt.tight_layout()
plt.show()

# Trouver le nombre de composantes pour 80% et 90% de variance
n_components_80 = # VOTRE CODE
n_components_90 = # VOTRE CODE

print(f"\nRésultats :")
print(f"  • Les 3 premières composantes expliquent {cumulative_variance[2]*100:.2f}% de la variance")
print(f"  • Il faut {n_components_80} composantes pour expliquer >80% de la variance")
print(f"  • Il faut {n_components_90} composantes pour expliquer >90% de la variance")

### 5.3 PCA à 3 composantes pour visualisation

Nous allons projeter nos données sur 3 composantes principales pour pouvoir les visualiser en 3D, de plus elles expliquent quasiment 90% de la variance.

In [ ]:
# PCA à 3 composantes
pca_3d = # VOTRE CODE
data_proj = # VOTRE CODE

print(f"PCA effectuée : {data_scaled.shape[1]} dimensions → 3 composantes principales")
print(f"\nVariance expliquée par les 3 composantes :")
for i, var in enumerate(pca_3d.explained_variance_ratio_, 1):
    print(f"  PC{i} : {var*100:.2f}%")
print(f"\nVariance totale expliquée : {pca_3d.explained_variance_ratio_.sum()*100:.2f}%")

# Création d'un DataFrame pour faciliter l'analyse
df_pca = pd.DataFrame(data_proj, columns=['PC1', 'PC2', 'PC3'])
print(f"\nDimensions des données projetées : {df_pca.shape}")

### 5.4 Visualisation des données dans l'espace PCA

In [ ]:
# Visualisation 3D dans l'espace PCA
fig = # VOTRE CODE
fig.update_traces(marker=dict(size=3, color='steelblue'))
fig.show()

print("\nObservation : La projection PCA révèle une structure plus étalée des données.")
print("Les composantes principales capturent les directions de variance maximale.")

### 5.5 Analyse des composantes principales

Voyons quelles features originales contribuent le plus à chaque composante principale.

In [ ]:
# Création d'un DataFrame des composantes
components_df = pd.DataFrame(
    pca_3d.components_.T,
    columns=['PC1', 'PC2', 'PC3'],
    index=data_num.columns
)

# Visualisation des contributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, pc in enumerate(['PC1', 'PC2', 'PC3']):
    # Trier par valeur absolue pour voir les contributions importantes
    sorted_features = components_df[pc].abs().sort_values(ascending=True)
    colors = ['red' if x < 0 else 'steelblue' for x in components_df.loc[sorted_features.index, pc]]
    
    axes[i].barh(range(len(sorted_features)), 
                 components_df.loc[sorted_features.index, pc],
                 color=colors, alpha=0.7)
    axes[i].set_yticks(range(len(sorted_features)))
    axes[i].set_yticklabels(sorted_features.index)
    axes[i].set_xlabel('Contribution', fontsize=11)
    axes[i].set_title(f'{pc}\n({pca_3d.explained_variance_ratio_[i]*100:.1f}% variance)', 
                     fontsize=12, fontweight='bold')
    axes[i].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    axes[i].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nInterprétation des composantes principales :")
print("\nPC1 - Principales contributions :")
top_pc1 = components_df['PC1'].abs().sort_values(ascending=False).head(3)
for feat, val in top_pc1.items():
    print(f"  • {feat}: {components_df.loc[feat, 'PC1']:.3f}")

print("\nPC2 - Principales contributions :")
top_pc2 = components_df['PC2'].abs().sort_values(ascending=False).head(3)
for feat, val in top_pc2.items():
    print(f"  • {feat}: {components_df.loc[feat, 'PC2']:.3f}")

print("\nPC3 - Principales contributions :")
top_pc3 = components_df['PC3'].abs().sort_values(ascending=False).head(3)
for feat, val in top_pc3.items():
    print(f"  • {feat}: {components_df.loc[feat, 'PC3']:.3f}")

## 6. K-means optimisé sur les composantes principales

### 6.1 Clustering dans l'espace PCA

In [ ]:
# K-means sur les données projetées PCA

# VOTRE CODE

### 6.2 Visualisation du clustering dans l'espace PCA

In [ ]:
# Visualisation avec les clusters dans l'espace PCA

# VOTRE CODE

### 6.3 Retour aux features originales

Visualisons maintenant ces clusters dans l'espace des features originales que vous avez choisies

In [ ]:
# Visualisation des clusters PCA dans l'espace original
df_original_clustered = data_num[# les features que vous avez choisies au départ ].copy()
df_original_clustered['cluster'] = labels_pca.astype(str)

# VOTRE CODE (visualisation)

## 7. Détermination du nombre optimal de clusters

### 7.1 Méthode du coude (Elbow method)

Testons différents nombres de clusters et analysons l'inertie (somme des distances au carré aux centroïdes).

In [ ]:
# Test de différents nombres de clusters

# VOTRE CODE

### 7.2 Visualisation de la courbe du coude

In [ ]:
# Graphique de l'inertie
plt.figure(figsize=(12, 6))
plt.plot(K_range, inertias, 'o-', linewidth=2, markersize=8, color='steelblue')
plt.xlabel('Nombre de clusters (k)', fontsize=12)
plt.ylabel('Inertie (somme des distances²)', fontsize=12)
plt.title('Méthode du coude pour déterminer le nombre optimal de clusters', fontsize=14, pad=20)
plt.grid(True, alpha=0.3)
plt.xticks(K_range)

# Marquer la position que nous avions choisie arbitrairement
plt.axvline(x=k_empirical, color='red', linestyle='--', alpha=0.7, 
            label=f'Suggestion arbitraire (k={k_empirical})')
plt.legend()

plt.tight_layout()
plt.show()

print("\nAnalyse de la courbe du coude :")
print("Le 'coude' représente le point où ajouter des clusters supplémentaires")
print("n'apporte plus d'amélioration significative.")
print("\nQuestion : Quel nombre de clusters vous semble optimal ?")

### 7.3 Calcul du taux de décroissance de l'inertie

In [ ]:
# VOTRE CODE

### 7.4 Clustering final avec le nombre optimal de clusters

In [ ]:
# Choisir le nombre final de clusters (vous pouvez modifier cette valeur)
k_final = # VOTRE CHOIX

print(f"Nombre final de clusters choisi : {k_final}")
print("\nEntraînement du modèle K-means final...")

# K-means final
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
labels_final = kmeans_final.fit_predict(data_proj)

print(f"\nClustering final effectué avec {k_final} clusters")
print(f"Inertie finale : {kmeans_final.inertia_:.2f}")
print(f"\nRépartition finale des morceaux par cluster :")
unique, counts = np.unique(labels_final, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"  Cluster {cluster_id} : {count} morceaux ({count/len(labels_final)*100:.1f}%)")

### 7.5 Visualisation finale du clustering optimisé

In [ ]:
# Visualisation dans l'espace PCA avec le clustering final

# VOTRE CODE

## 8. Création de playlists personnalisées

### 8.1 Analyse des caractéristiques de chaque cluster

In [ ]:
# Ajouter les labels de cluster au DataFrame original (nettoyé)

# VOTRE CODE

# Calculer les moyennes des features pour chaque cluster (faire un groupby)
# VOTRE CODE

### 8.2 Visualisation des profils de clusters

In [ ]:
# Sélection de features clés pour la visualisation
key_features = ['danceability', 'energy', 'valence', 'acousticness', 'instrumentalness', 'tempo']

# Normalisation pour la visualisation en radar
cluster_profiles_norm = cluster_profiles[key_features].copy()
for col in key_features:
    if col == 'tempo':
        # Normaliser tempo sur [0, 1]
        cluster_profiles_norm[col] = (cluster_profiles_norm[col] - cluster_profiles_norm[col].min()) / \
                                      (cluster_profiles_norm[col].max() - cluster_profiles_norm[col].min())

# Heatmap des profils de clusters
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_profiles_norm.T, annot=True, fmt='.2f', cmap='YlOrRd', 
            cbar_kws={'label': 'Valeur normalisée'}, linewidths=0.5)
plt.title(f'Profils des {k_final} clusters musicaux', fontsize=14, pad=20)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Features audio', fontsize=12)
plt.tight_layout()
plt.show()

print("\nInterprétation : Chaque cluster a un profil musical distinct.")

### 8.3 Caractérisation automatique des clusters

In [ ]:
# Fonction pour caractériser un cluster
def characterize_cluster(cluster_id, profile):
    """Génère une description textuelle d'un cluster basée sur ses caractéristiques."""
    characteristics = []
    
    # Danceability
    if profile['danceability'] > 0.7:
        characteristics.append("très dansant")
    elif profile['danceability'] > 0.5:
        characteristics.append("dansant")
    else:
        characteristics.append("peu dansant")
    
    # Energy
    if profile['energy'] > 0.7:
        characteristics.append("énergique")
    elif profile['energy'] > 0.5:
        characteristics.append("modérément énergique")
    else:
        characteristics.append("calme")
    
    # Valence
    if profile['valence'] > 0.6:
        characteristics.append("joyeux")
    elif profile['valence'] > 0.4:
        characteristics.append("neutre")
    else:
        characteristics.append("mélancolique")
    
    # Acousticness
    if profile['acousticness'] > 0.6:
        characteristics.append("acoustique")
    elif profile['acousticness'] < 0.3:
        characteristics.append("électronique")
    
    # Instrumentalness
    if profile['instrumentalness'] > 0.5:
        characteristics.append("instrumental")
    
    # Tempo
    if profile['tempo'] > 140:
        characteristics.append("rapide")
    elif profile['tempo'] < 90:
        characteristics.append("lent")
    
    return ", ".join(characteristics)

# Générer les descriptions pour chaque cluster
print("\n🎼 Caractérisation des clusters musicaux :\n")
print("="*80)
for cluster_id in range(k_final):
    profile = cluster_profiles.loc[cluster_id]
    description = characterize_cluster(cluster_id, profile)
    n_songs = (labels_final == cluster_id).sum()
    
    print(f"\nCluster {cluster_id} ({n_songs} morceaux)")
    print(f"   Style : {description.capitalize()}")
    print(f"   Caractéristiques :")
    print(f"     • Danceability : {profile['danceability']:.2f}")
    print(f"     • Energy       : {profile['energy']:.2f}")
    print(f"     • Valence      : {profile['valence']:.2f}")
    print(f"     • Tempo        : {profile['tempo']:.0f} BPM")

print("\n" + "="*80)

### 8.4 Génération de playlists thématiques

In [ ]:
# Fonction pour créer une playlist à partir d'un cluster
def create_playlist(cluster_id, n_songs=10, seed=None):
    """Crée une playlist de n_songs morceaux du cluster spécifié."""
    cluster_songs = df_with_clusters[df_with_clusters['cluster'] == cluster_id]
    
    if seed is not None:
        np.random.seed(seed)
    
    n_songs = min(n_songs, len(cluster_songs))
    playlist = # VOTRE CODE
    
    return playlist[['track_name', 'artists', 'danceability', 'energy', 'valence', 'tempo']]

# Créer des playlists pour quelques clusters
print("\nExemples de playlists générées :\n")
print("="*80)

for cluster_id in range(min(3, k_final)):  # Afficher les 3 premiers clusters
    print(f"\nPlaylist du Cluster {cluster_id}")
    playlist = create_playlist(cluster_id, n_songs=5, seed=42)
    print(playlist.to_string(index=False))
    print("\n" + "-"*80)

print("\n" + "="*80)

### 8.5 Système de recommandation basé sur un morceau

Créons une fonction qui recommande des morceaux similaires à un morceau donné.

In [ ]:
def recommend_similar_songs(track_name, n_recommendations=10):
    """Recommande des morceaux similaires basés sur le même cluster."""
    # Trouver le morceau
    # VOTRE CODE
    
    # Prendre le premier match
    # VOTRE CODE
    
    # Trouver d'autres morceaux du même cluster
    # VOTRE CODE
    
    # Calculer la distance dans l'espace PCA et les ordonner pour affiner
    # VOTRE CODE
    
    # Retourner les n recommandations
    # VOTRE CODE
    
    return recommendations

# Exemple d'utilisation
print("\n" + "="*80)
print("Exemple de recommandations")
print("="*80)

# Choisir un morceau aléatoire pour la démonstration
random_track = df['track_name'].sample(1, random_state=42).values[0]
recommendations = recommend_similar_songs(random_track, n_recommendations=8)

### 8.6 Export des playlists

Exportons les playlists pour chaque cluster dans des fichiers CSV séparés.

In [ ]:
# Créer un répertoire pour les playlists
import os
os.makedirs('playlists', exist_ok=True)

# Exporter chaque cluster dans un fichier CSV
for cluster_id in range(k_final):
    cluster_songs = df_with_clusters[df_with_clusters['cluster'] == cluster_id]
    profile = cluster_profiles.loc[cluster_id]
    description = characterize_cluster(cluster_id, profile)
    
    filename = f'playlists/cluster_{cluster_id}_{description.replace(",", "").replace(" ", "_")}.csv'
    cluster_songs.to_csv(filename, index=False)
    print(f"✓ Playlist du Cluster {cluster_id} exportée : {filename}")

print(f"\n✓ {k_final} playlists exportées dans le dossier 'playlists/'")

## 9. Conclusion et perspectives

Bien sûr nous ne sommes pas au niveau des équipes qui ont conçu le système de recommandation de Spotify, Deezer ou de Youtube. Notre échantillon était réduit et nous nous sommes volontairement limité dans notre clustering etc. mais la logique était là.

### Ce que vous avez appris

1. **K-means sur données brutes** : Performance limitée car les features ont des échelles différentes

2. **Mise à l'échelle** : Amélioration modeste mais importante pour égaliser l'influence des features

3. **PCA + K-means** : Combinaison puissante qui :
   - Réduit la dimensionnalité tout en conservant l'information essentielle
   - Révèle la structure sous-jacente des données
   - Améliore significativement la qualité du clustering

4. **Méthode du coude** : Permet de déterminer (plus ou moins) objectivement le nombre optimal de clusters. Notre tentative d’automatiser complètement le processus de sélection via le taux d’accroissement et le calcul de la dérivé seconde ne fonctionnait pas totalement dans ce contexte limité, mais elle est à connaître.

### Applications pratiques

Ce système de recommandation peut être utilisé pour :
- Générer des playlists automatiques cohérentes
- Suggérer de nouveaux morceaux similaires aux goûts d'un utilisateur
- Organiser une bibliothèque musicale par style/ambiance
- Créer des transitions fluides dans des DJ sets

### Pour aller plus loin

1. **Tester d'autres algorithmes** : DBSCAN, Hierarchical Clustering, GMM
2. **Incorporer des données supplémentaires** : genre musical, année de sortie, paroles
3. **Utiliser des métriques d'évaluation** : Silhouette Score, Davies-Bouldin Index
4. **Créer un système hybride** : combiner clustering et collaborative filtering
5. **Interface utilisateur** : développer une application web interactive

### Exercices complémentaires

1. Comparez les résultats avec différents scalers (StandardScaler, MinMaxScaler)
2. Testez la PCA avec un nombre différent de composantes
3. Analysez les morceaux mal classés (outliers)
4. Créez des playlists thématiques (sport, détente, fête, travail)
5. Implémentez une fonction de "découverte" qui suggère des morceaux de clusters adjacents

## 10. À vous de jouer !

Utilisez les cellules ci-dessous pour expérimenter et créer vos propres playlists personnalisées.

In [ ]:
# Cellule d'expérimentation libre
# Testez vos propres recommandations ici !

# Exemple : recommander_similar_songs("votre_morceau_préféré", n_recommendations=10)

In [ ]:
# Créez votre propre playlist personnalisée en mélangeant plusieurs clusters
# Exemple : combiner des morceaux énergiques et joyeux

# custom_playlist = pd.concat([
#     create_playlist(cluster_1, n_songs=5),
#     create_playlist(cluster_2, n_songs=5)
# ])
# print(custom_playlist)